# Image Segmentation


Computer vision is a rapidly growing field with new advances occurring daily. One of the coolest abilities of computer vision is being able to detect individual objects in an image or video.

Images are simply made up of pixels which can be "segmented" into objects by a machine learning model. This model learns the classes of the desired classifications and then is able to find them in the image.

Semantic Segmentation refers to classifying objects with the same pixel values inside the same colourmaps. This differs from instance segmentation where different instances of the same object are given differing colourmaps

# Usage

The setup for using image segmentation is simple using OpenCV and Pixellib on Python. Simply import the necessary dependencies:

In [6]:
try:
  import pixellib
except ModuleNotFoundError:
  !pip install pixellib
  import pixellib
import numpy as np
import cv2
import os
import time
from pixellib.semantic import semantic_segmentation
from __future__ import print_function
from ipywidgets import interact
from IPython.display import Image
import ipywidgets as widgets

Next we select the image we'd like to pass to to the model (1 - Puppy, 2 - Cat, 3 - Cat AND Puppy)

In [10]:
!ls
@interact
def show_image_options(column = ["Puppy", "Cat", "Car"], file=os.listdir('images/')):
  display(Image(fdir+file))
  return column

sample_data


FileNotFoundError: ignored